In [1]:
import numpy as np
import pickle

import os
import sys
sys.path.append('../Parameters/')
sys.path.append('../AnalyticalModel/')

from Metadata import read_metadata
import ParametersFromLiterature as prms

from FullMatrix import FullStates
from AnalyticalSimulationRun import get_steady_states, get_means, get_means_and_vars, get_correlation_times

In [2]:
def read_files_shift_compensation(file):
    shift_compensation = None
    try:
        f = open(file+'_shift_compensation', 'rb')
        shift_compensation = pickle.load(f)
        f.close()
    except:
        print('Shift compensation could not be loaded.')
    return shift_compensation
        
def read_files_steady_state(file, grid_prms, varied_prms):
    steady_states      = None
    probabilities      = None
    try:
        f = open(file+'_steady_states', 'rb')
        steady_states = pickle.load(f)
        f.close()

        probabilities = []
        S = FullStates(grid_prms.L)
        sites = grid_prms.sites.flatten()
        for J, eb in zip(varied_prms.J_list, varied_prms.eb_list):
            probabilities.append([])
            for htf in varied_prms.htf_list:
                probabilities[-1].append(S.compact_states(steady_states[f'J{J}eb{eb}htf{htf}'][1], sites))
        probabilities = np.array(probabilities)
    except:
        print('Steady states could not be loaded.')
    return steady_states, probabilities

def read_files_means(file, varied_prms):
    means_dict         = None
    means              = None
    try:
        f = open(file+'_means', 'rb')
        means_dict = pickle.load(f)
        f.close()

        means = []
        for J, eb in zip(varied_prms.J_list, varied_prms.eb_list):
            means.append([])
            for htf in varied_prms.htf_list:
                means[-1].append(means_dict[f'J{J}eb{eb}htf{htf}'])
        means = np.array(means)
    except:
        print('Means could not be loaded.')
    return means_dict, means

def read_files_means_and_vars(file, varied_prms):
    means_and_vars     = None
    means              = None
    stds               = None
    try:
        f = open(file+'_means_and_vars', 'rb')
        means_and_vars = pickle.load(f)
        f.close()

        means = []
        stds  = []
        for J, eb in zip(varied_prms.J_list, varied_prms.eb_list):
            means.append([])
            stds.append([])
            for htf in varied_prms.htf_list:
                mu, sigma_squared = means_and_vars[f'J{J}eb{eb}htf{htf}']
                means[-1].append(mu)
                stds[-1].append(np.sqrt(sigma_squared))
        means = np.array(means)
        stds  = np.array(stds)
    except:
        print('Means and vars could not be loaded.')
    return means_and_vars, means, stds

def read_files_correlation_times(file, varied_prms):
    correlation_times  = None
    taus               = None
    try:
        f = open(file+'_correlation_times', 'rb')
        correlation_times = pickle.load(f)
        f.close()

        taus = []
        for J, eb in zip(varied_prms.J_list, varied_prms.eb_list):
            taus.append([])
            for htf in varied_prms.htf_list:
                taus[-1].append(correlation_times[f'J{J}eb{eb}htf{htf}'])
        taus = np.array(taus)
    except:
        print('Correlation times could not be loaded.')
    return correlation_times, taus

def read_files(file): 
    grid_prms, sim_prms, varied_prms = read_metadata(file)
    shift_compensation               = read_files_shift_compensation(file)
    steady_states, probabilities     = read_files_steady_state(file, grid_prms, varied_prms)
    means_and_vars, means, stds      = read_files_means_and_vars(file, varied_prms)
    correlation_times, taus          = read_files_correlation_times(file, varied_prms)
    
    means_dict = None
    if np.any(means == None):
        means_dict, means            = read_files_means(file, varied_prms)
        
    return grid_prms, sim_prms, varied_prms, shift_compensation, steady_states, probabilities, means_dict, \
           means, means_and_vars, stds, correlation_times, taus 

*Read files:*

In [ ]:
file_names = ['StochasticModel/6Sites_NoPadding_NonPeriodic_SteadyState_InfiniteBath_Occupancy_2024-04-30',\
              'StochasticModel/6Sites_2Padding_NonPeriodic_SteadyState_InfiniteBath_Occupancy_2024-04-30',\
              'StochasticModel/6Sites_NoPadding_NonPeriodic_SteadyState_InfiniteBath_N6Expression_2024-04-30',\
              'StochasticModel/6Sites_2Padding_NonPeriodic_SteadyState_InfiniteBath_N6Expression_2024-04-30',\
              '6Sites_NoPadding_NonPeriodic_SteadyState_InfiniteBath_Occupancy_2025-05-09',\
              '6Sites_2Padding_NonPeriodic_SteadyState_InfiniteBath_Occupancy_2025-05-09']

## Choose file to load:
file_name = file_names[0]
file = os.path.join(os.path.realpath('__file__'), f'../../Data/{file_name}')

grid_prms, sim_prms, varied_prms, shift_compensation, steady_states, probabilities, means_dict, \
means, means_and_vars, stds, correlation_times, taus = read_files(file)

*Create files:*

In [ ]:
get_steady_states(varied_prms.htf_list, varied_prms.J_list, varied_prms.eb_list, grid_prms, file=file)

J=0.0, eb=14.143319868324545, htf=-19.172368036976454
J=0.0, eb=14.143319868324545, htf=-18.073755738876944
J=0.0, eb=14.143319868324545, htf=-17.56293010567955
J=0.0, eb=14.143319868324545, htf=-17.22645785962693
J=0.0, eb=14.143319868324545, htf=-16.97514342191462
J=0.0, eb=14.143319868324545, htf=-16.774472717021062
J=0.0, eb=14.143319868324545, htf=-16.60741862292649
J=0.0, eb=14.143319868324545, htf=-16.464317769854414
J=0.0, eb=14.143319868324545, htf=-16.339154617469003
J=0.0, eb=14.143319868324545, htf=-16.227928972927376
J=0.0, eb=14.143319868324545, htf=-16.12784550493899
J=0.0, eb=14.143319868324545, htf=-16.036873717301855
J=0.0, eb=14.143319868324545, htf=-15.9534920989314
J=0.0, eb=14.143319868324545, htf=-15.876531048363866
J=0.0, eb=14.143319868324545, htf=-15.805072074950317
J=0.0, eb=14.143319868324545, htf=-15.73838069102024
J=0.0, eb=14.143319868324545, htf=-15.6758603246075
J=0.0, eb=14.143319868324545, htf=-15.61701981515316
J=0.0, eb=14.143319868324545, htf=-15.5

J=0.0, eb=14.143319868324545, htf=-13.400925404820649
J=0.0, eb=14.143319868324545, htf=-13.39471419529659
J=0.0, eb=14.143319868324545, htf=-13.38854132675809
J=0.0, eb=14.143319868324545, htf=-13.382406328759156
J=0.0, eb=14.143319868324545, htf=-13.376308739459619
J=0.0, eb=14.143319868324545, htf=-13.37024810541651
J=0.0, eb=14.143319868324545, htf=-13.36422398138171
J=0.0, eb=14.143319868324545, htf=-13.358235930105668
J=0.0, eb=14.143319868324545, htf=-13.352283522146953
J=0.0, eb=14.143319868324545, htf=-13.346366335687446
J=0.0, eb=14.143319868324545, htf=-13.34048395635296
J=0.0, eb=14.143319868324545, htf=-13.334635977039119
J=0.0, eb=14.143319868324545, htf=-13.32882199774228
J=0.0, eb=14.143319868324545, htf=-13.32304162539536
J=0.0, eb=14.143319868324545, htf=-13.317294473708372
J=0.0, eb=14.143319868324545, htf=-13.311580163013515
J=0.0, eb=14.143319868324545, htf=-13.305898320114663
J=0.0, eb=14.143319868324545, htf=-13.300248578141124
J=0.0, eb=14.143319868324545, htf=-

J=0.0, eb=14.143319868324545, htf=-12.712460573976495
J=0.0, eb=14.143319868324545, htf=-12.709335562001927
J=0.0, eb=14.143319868324545, htf=-12.706220285253544
J=0.0, eb=14.143319868324545, htf=-12.703114683263959
J=0.0, eb=14.143319868324545, htf=-12.700018696127398
J=0.0, eb=14.143319868324545, htf=-12.696932264492775
J=0.0, eb=14.143319868324545, htf=-12.693855329556863
J=0.0, eb=14.143319868324545, htf=-12.690787833057568
J=0.0, eb=14.143319868324545, htf=-12.687729717267315
J=0.0, eb=14.143319868324545, htf=-12.684680924986523
J=0.0, eb=14.143319868324545, htf=-12.681641399537192
J=0.0, eb=14.143319868324545, htf=-12.67861108475658
J=0.0, eb=14.143319868324545, htf=-12.67558992499097
J=0.0, eb=14.143319868324545, htf=-12.672577865089545
J=0.0, eb=14.143319868324545, htf=-12.669574850398341
J=0.0, eb=14.143319868324545, htf=-12.666580826754304
J=0.0, eb=14.143319868324545, htf=-12.663595740479417
J=0.0, eb=14.143319868324545, htf=-12.660619538374936
J=0.0, eb=14.143319868324545, 

J=0.0, eb=14.143319868324545, htf=-12.312748643963834
J=0.0, eb=14.143319868324545, htf=-12.31065219770586
J=0.0, eb=14.143319868324545, htf=-12.30856013730219
J=0.0, eb=14.143319868324545, htf=-12.306472444440258
J=0.0, eb=14.143319868324545, htf=-12.304389100921957
J=0.0, eb=14.143319868324545, htf=-12.30231008866268
J=0.0, eb=14.143319868324545, htf=-12.300235389690371
J=0.0, eb=14.143319868324545, htf=-12.298164986144615
J=0.0, eb=14.143319868324545, htf=-12.296098860275697
J=0.0, eb=14.143319868324545, htf=-12.29403699444369
J=0.0, eb=14.143319868324545, htf=-12.291979371117563
J=0.0, eb=14.143319868324545, htf=-12.289925972874274
J=0.0, eb=14.143319868324545, htf=-12.287876782397891
J=0.0, eb=14.143319868324545, htf=-12.285831782478716
J=0.0, eb=14.143319868324545, htf=-12.283790956012416
J=0.0, eb=14.143319868324545, htf=-12.281754285999165
J=0.0, eb=14.143319868324545, htf=-12.279721755542795
J=0.0, eb=14.143319868324545, htf=-12.277693347849954
J=0.0, eb=14.143319868324545, ht

J=0.0, eb=14.143319868324545, htf=-12.023230440028065
J=0.0, eb=14.143319868324545, htf=-12.021660571561483
J=0.0, eb=14.143319868324545, htf=-12.02009316369001
J=0.0, eb=14.143319868324545, htf=-12.018528208712246
J=0.0, eb=14.143319868324545, htf=-12.016965698962894
J=0.0, eb=14.143319868324545, htf=-12.01540562681253
J=0.0, eb=14.143319868324545, htf=-12.013847984667386
J=0.0, eb=14.143319868324545, htf=-12.012292764969118
J=0.0, eb=14.143319868324545, htf=-12.010739960194597
J=0.0, eb=14.143319868324545, htf=-12.00918956285568
J=0.0, eb=14.143319868324545, htf=-12.007641565499004
J=1.0, eb=13.289575767386253, htf=-19.172368036976454
J=1.0, eb=13.289575767386253, htf=-18.073755738876944
J=1.0, eb=13.289575767386253, htf=-17.56293010567955
J=1.0, eb=13.289575767386253, htf=-17.22645785962693
J=1.0, eb=13.289575767386253, htf=-16.97514342191462
J=1.0, eb=13.289575767386253, htf=-16.774472717021062
J=1.0, eb=13.289575767386253, htf=-16.60741862292649
J=1.0, eb=13.289575767386253, htf=-

...

J=14.0, eb=2.143424269595327, htf=-13.36422398138171
J=14.0, eb=2.143424269595327, htf=-13.358235930105668
J=14.0, eb=2.143424269595327, htf=-13.352283522146953
J=14.0, eb=2.143424269595327, htf=-13.346366335687446
J=14.0, eb=2.143424269595327, htf=-13.34048395635296
J=14.0, eb=2.143424269595327, htf=-13.334635977039119
J=14.0, eb=2.143424269595327, htf=-13.32882199774228
J=14.0, eb=2.143424269595327, htf=-13.32304162539536
J=14.0, eb=2.143424269595327, htf=-13.317294473708372
J=14.0, eb=2.143424269595327, htf=-13.311580163013515
J=14.0, eb=2.143424269595327, htf=-13.305898320114663
J=14.0, eb=2.143424269595327, htf=-13.300248578141124
J=14.0, eb=2.143424269595327, htf=-13.294630576405481
J=14.0, eb=2.143424269595327, htf=-13.289043960265422
J=14.0, eb=2.143424269595327, htf=-13.2834883809894
J=14.0, eb=2.143424269595327, htf=-13.27796349562601
J=14.0, eb=2.143424269595327, htf=-13.272468966876948
J=14.0, eb=2.143424269595327, htf=-13.26700446297345
J=14.0, eb=2.143424269595327, htf=-1

J=14.0, eb=2.143424269595327, htf=-12.696932264492775
J=14.0, eb=2.143424269595327, htf=-12.693855329556863
J=14.0, eb=2.143424269595327, htf=-12.690787833057568
J=14.0, eb=2.143424269595327, htf=-12.687729717267315
J=14.0, eb=2.143424269595327, htf=-12.684680924986523
J=14.0, eb=2.143424269595327, htf=-12.681641399537192
J=14.0, eb=2.143424269595327, htf=-12.67861108475658
J=14.0, eb=2.143424269595327, htf=-12.67558992499097
J=14.0, eb=2.143424269595327, htf=-12.672577865089545
J=14.0, eb=2.143424269595327, htf=-12.669574850398341
J=14.0, eb=2.143424269595327, htf=-12.666580826754304
J=14.0, eb=2.143424269595327, htf=-12.663595740479417
J=14.0, eb=2.143424269595327, htf=-12.660619538374936
J=14.0, eb=2.143424269595327, htf=-12.6576521677157
J=14.0, eb=2.143424269595327, htf=-12.65469357624452
J=14.0, eb=2.143424269595327, htf=-12.651743712166667
J=14.0, eb=2.143424269595327, htf=-12.648802524144415
J=14.0, eb=2.143424269595327, htf=-12.645869961291703
J=14.0, eb=2.143424269595327, htf

J=14.0, eb=2.143424269595327, htf=-12.30856013730219
J=14.0, eb=2.143424269595327, htf=-12.306472444440258
J=14.0, eb=2.143424269595327, htf=-12.304389100921957
J=14.0, eb=2.143424269595327, htf=-12.30231008866268
J=14.0, eb=2.143424269595327, htf=-12.300235389690371
J=14.0, eb=2.143424269595327, htf=-12.298164986144615
J=14.0, eb=2.143424269595327, htf=-12.296098860275697
J=14.0, eb=2.143424269595327, htf=-12.29403699444369
J=14.0, eb=2.143424269595327, htf=-12.291979371117563
J=14.0, eb=2.143424269595327, htf=-12.289925972874274
J=14.0, eb=2.143424269595327, htf=-12.287876782397891
J=14.0, eb=2.143424269595327, htf=-12.285831782478716
J=14.0, eb=2.143424269595327, htf=-12.283790956012416
J=14.0, eb=2.143424269595327, htf=-12.281754285999165
J=14.0, eb=2.143424269595327, htf=-12.279721755542795
J=14.0, eb=2.143424269595327, htf=-12.277693347849954
J=14.0, eb=2.143424269595327, htf=-12.275669046229275
J=14.0, eb=2.143424269595327, htf=-12.273648834090551
J=14.0, eb=2.143424269595327, h

J=14.0, eb=2.143424269595327, htf=-12.031116963392282
J=14.0, eb=2.143424269595327, htf=-12.029534675149693
J=14.0, eb=2.143424269595327, htf=-12.027954886558714
J=14.0, eb=2.143424269595327, htf=-12.026377589734006
J=14.0, eb=2.143424269595327, htf=-12.024802776827478
J=14.0, eb=2.143424269595327, htf=-12.023230440028065
J=14.0, eb=2.143424269595327, htf=-12.021660571561483
J=14.0, eb=2.143424269595327, htf=-12.02009316369001
J=14.0, eb=2.143424269595327, htf=-12.018528208712246
J=14.0, eb=2.143424269595327, htf=-12.016965698962894
J=14.0, eb=2.143424269595327, htf=-12.01540562681253
J=14.0, eb=2.143424269595327, htf=-12.013847984667386
J=14.0, eb=2.143424269595327, htf=-12.012292764969118
J=14.0, eb=2.143424269595327, htf=-12.010739960194597
J=14.0, eb=2.143424269595327, htf=-12.00918956285568
J=14.0, eb=2.143424269595327, htf=-12.007641565499004


{'J0.0eb14.143319868324545htf-19.172368036976454': (array(['0000000000', '0000000001', '0000000010', ..., '1111111101',
         '1111111110', '1111111111'], dtype='<U10'),
  array([9.61613873e-01, 4.53468450e-09, 4.53468450e-09, ...,
         7.92706787e-39, 7.92706787e-39, 3.73816901e-47])),
 'J0.0eb14.143319868324545htf-18.073755738876944': (array(['0000000000', '0000000001', '0000000010', ..., '1111111101',
         '1111111110', '1111111111'], dtype='<U10'),
  array([8.89879661e-01, 1.25892221e-08, 1.25892221e-08, ...,
         1.44389119e-34, 1.44389119e-34, 2.04268821e-42])),
 'J0.0eb14.143319868324545htf-17.56293010567955': (array(['0000000000', '0000000001', '0000000010', ..., '1111111101',
         '1111111110', '1111111111'], dtype='<U10'),
  array([8.24311470e-01, 1.94360368e-08, 1.94360368e-08, ...,
         1.32719065e-32, 1.32719065e-32, 3.12931788e-40])),
 'J0.0eb14.143319868324545htf-17.22645785962693': (array(['0000000000', '0000000001', '0000000010', ..., '1111111101

In [ ]:
get_means(varied_prms.htf_list, varied_prms.J_list, varied_prms.eb_list, grid_prms, sim_prms.T, N=shift_compensation.N_expr, file=file)

In [ ]:
get_means_and_vars(varied_prms.htf_list, varied_prms.J_list, varied_prms.eb_list, grid_prms, sim_prms.T, N=shift_compensation.N_expr, file=file)

J=0.0, eb=14.143319868324545, htf=-19.172368036976454
J=0.0, eb=14.143319868324545, htf=-18.073755738876944
J=0.0, eb=14.143319868324545, htf=-17.56293010567955
J=0.0, eb=14.143319868324545, htf=-17.22645785962693
J=0.0, eb=14.143319868324545, htf=-16.97514342191462
J=0.0, eb=14.143319868324545, htf=-16.774472717021062
J=0.0, eb=14.143319868324545, htf=-16.60741862292649
J=0.0, eb=14.143319868324545, htf=-16.464317769854414
J=0.0, eb=14.143319868324545, htf=-16.339154617469003
J=0.0, eb=14.143319868324545, htf=-16.227928972927376
J=0.0, eb=14.143319868324545, htf=-16.12784550493899
J=0.0, eb=14.143319868324545, htf=-16.036873717301855
J=0.0, eb=14.143319868324545, htf=-15.9534920989314
J=0.0, eb=14.143319868324545, htf=-15.876531048363866
J=0.0, eb=14.143319868324545, htf=-15.805072074950317
J=0.0, eb=14.143319868324545, htf=-15.73838069102024
J=0.0, eb=14.143319868324545, htf=-15.6758603246075
J=0.0, eb=14.143319868324545, htf=-15.61701981515316
J=0.0, eb=14.143319868324545, htf=-15.5

J=0.0, eb=14.143319868324545, htf=-13.445518846383393
J=0.0, eb=14.143319868324545, htf=-13.439025307641428
J=0.0, eb=14.143319868324545, htf=-13.43257366292852
J=0.0, eb=14.143319868324545, htf=-13.426163375136184
J=0.0, eb=14.143319868324545, htf=-13.419793917419286
J=0.0, eb=14.143319868324545, htf=-13.413464772936221
J=0.0, eb=14.143319868324545, htf=-13.407175434597239
J=0.0, eb=14.143319868324545, htf=-13.400925404820649
J=0.0, eb=14.143319868324545, htf=-13.39471419529659
J=0.0, eb=14.143319868324545, htf=-13.38854132675809
J=0.0, eb=14.143319868324545, htf=-13.382406328759156
J=0.0, eb=14.143319868324545, htf=-13.376308739459619
J=0.0, eb=14.143319868324545, htf=-13.37024810541651
J=0.0, eb=14.143319868324545, htf=-13.36422398138171
J=0.0, eb=14.143319868324545, htf=-13.358235930105668
J=0.0, eb=14.143319868324545, htf=-13.352283522146953
J=0.0, eb=14.143319868324545, htf=-13.346366335687446
J=0.0, eb=14.143319868324545, htf=-13.34048395635296
J=0.0, eb=14.143319868324545, htf=

J=0.0, eb=14.143319868324545, htf=-12.754000215026432
J=0.0, eb=14.143319868324545, htf=-12.750742873724693
J=0.0, eb=14.143319868324545, htf=-12.747496108194392
J=0.0, eb=14.143319868324545, htf=-12.744259849983752
J=0.0, eb=14.143319868324545, htf=-12.741034031303439
J=0.0, eb=14.143319868324545, htf=-12.737818585018033
J=0.0, eb=14.143319868324545, htf=-12.734613444637652
J=0.0, eb=14.143319868324545, htf=-12.731418544309701
J=0.0, eb=14.143319868324545, htf=-12.72823381881075
J=0.0, eb=14.143319868324545, htf=-12.725059203538544
J=0.0, eb=14.143319868324545, htf=-12.72189463450415
J=0.0, eb=14.143319868324545, htf=-12.718740048324202
J=0.0, eb=14.143319868324545, htf=-12.715595382213298
J=0.0, eb=14.143319868324545, htf=-12.712460573976495
J=0.0, eb=14.143319868324545, htf=-12.709335562001927
J=0.0, eb=14.143319868324545, htf=-12.706220285253544
J=0.0, eb=14.143319868324545, htf=-12.703114683263959
J=0.0, eb=14.143319868324545, htf=-12.700018696127398
J=0.0, eb=14.143319868324545, 

J=0.0, eb=14.143319868324545, htf=-12.349077585596927
J=0.0, eb=14.143319868324545, htf=-12.346903662265861
J=0.0, eb=14.143319868324545, htf=-12.344734454586872
J=0.0, eb=14.143319868324545, htf=-12.342569942145854
J=0.0, eb=14.143319868324545, htf=-12.340410104660979
J=0.0, eb=14.143319868324545, htf=-12.33825492198155
J=0.0, eb=14.143319868324545, htf=-12.336104374086876
J=0.0, eb=14.143319868324545, htf=-12.333958441085153
J=0.0, eb=14.143319868324545, htf=-12.331817103212364
J=0.0, eb=14.143319868324545, htf=-12.329680340831183
J=0.0, eb=14.143319868324545, htf=-12.327548134429897
J=0.0, eb=14.143319868324545, htf=-12.325420464621335
J=0.0, eb=14.143319868324545, htf=-12.323297312141811
J=0.0, eb=14.143319868324545, htf=-12.32117865785008
J=0.0, eb=14.143319868324545, htf=-12.319064482726297
J=0.0, eb=14.143319868324545, htf=-12.316954767871001
J=0.0, eb=14.143319868324545, htf=-12.314849494504093
J=0.0, eb=14.143319868324545, htf=-12.312748643963834
J=0.0, eb=14.143319868324545, 

J=0.0, eb=14.143319868324545, htf=-12.061666141961673
J=0.0, eb=14.143319868324545, htf=-12.06003481079813
J=0.0, eb=14.143319868324545, htf=-12.058406136511548
J=0.0, eb=14.143319868324545, htf=-12.056780110461672
J=0.0, eb=14.143319868324545, htf=-12.055156724050322
J=0.0, eb=14.143319868324545, htf=-12.05353596872113
J=0.0, eb=14.143319868324545, htf=-12.05191783595926
J=0.0, eb=14.143319868324545, htf=-12.050302317291147
J=0.0, eb=14.143319868324545, htf=-12.048689404284223
J=0.0, eb=14.143319868324545, htf=-12.04707908854666
J=0.0, eb=14.143319868324545, htf=-12.045471361727104
J=0.0, eb=14.143319868324545, htf=-12.043866215514416
J=0.0, eb=14.143319868324545, htf=-12.042263641637415
J=0.0, eb=14.143319868324545, htf=-12.040663631864621
J=0.0, eb=14.143319868324545, htf=-12.039066178004004
J=0.0, eb=14.143319868324545, htf=-12.03747127190273
J=0.0, eb=14.143319868324545, htf=-12.035878905446907
J=0.0, eb=14.143319868324545, htf=-12.034289070561346
J=0.0, eb=14.143319868324545, htf

...

J=14.0, eb=2.143424269595327, htf=-14.753527042492216
J=14.0, eb=2.143424269595327, htf=-14.729716384367089
J=14.0, eb=2.143424269595327, htf=-14.706459512771415
J=14.0, eb=2.143424269595327, htf=-14.68373125226245
J=14.0, eb=2.143424269595327, htf=-14.661508106046332
J=14.0, eb=2.143424269595327, htf=-14.639768109978519
J=14.0, eb=2.143424269595327, htf=-14.618490702099825
J=14.0, eb=2.143424269595327, htf=-14.597656605765575
J=14.0, eb=2.143424269595327, htf=-14.577247724702959
J=14.0, eb=2.143424269595327, htf=-14.557247048564882
J=14.0, eb=2.143424269595327, htf=-14.537638567745097
J=14.0, eb=2.143424269595327, htf=-14.5184071963858
J=14.0, eb=2.143424269595327, htf=-14.499538702650009
J=14.0, eb=2.143424269595327, htf=-14.481019645451363
J=14.0, eb=2.143424269595327, htf=-14.462837316936763
J=14.0, eb=2.143424269595327, htf=-14.444979690105347
J=14.0, eb=2.143424269595327, htf=-14.427435371023028
J=14.0, eb=2.143424269595327, htf=-14.410193555157115
J=14.0, eb=2.143424269595327, h

J=14.0, eb=2.143424269595327, htf=-13.208786863663944
J=14.0, eb=2.143424269595327, htf=-13.203658637865603
J=14.0, eb=2.143424269595327, htf=-13.198556576550287
J=14.0, eb=2.143424269595327, htf=-13.193480414087004
J=14.0, eb=2.143424269595327, htf=-13.188429888869514
J=14.0, eb=2.143424269595327, htf=-13.18340474323542
J=14.0, eb=2.143424269595327, htf=-13.178404723387292
J=14.0, eb=2.143424269595327, htf=-13.173429579315757
J=14.0, eb=2.143424269595327, htf=-13.168479064724478
J=14.0, eb=2.143424269595327, htf=-13.163552936957
J=14.0, eb=2.143424269595327, htf=-13.15865095692537
J=14.0, eb=2.143424269595327, htf=-13.153772889040516
J=14.0, eb=2.143424269595327, htf=-13.148918501144296
J=14.0, eb=2.143424269595327, htf=-13.144087564443208
J=14.0, eb=2.143424269595327, htf=-13.139279853443682
J=14.0, eb=2.143424269595327, htf=-13.134495145888923
J=14.0, eb=2.143424269595327, htf=-13.129733222697256
J=14.0, eb=2.143424269595327, htf=-13.124993867901937
J=14.0, eb=2.143424269595327, htf

J=14.0, eb=2.143424269595327, htf=-12.628452918709094
J=14.0, eb=2.143424269595327, htf=-12.625579344081926
J=14.0, eb=2.143424269595327, htf=-12.622714003177204
J=14.0, eb=2.143424269595327, htf=-12.61985684894499
J=14.0, eb=2.143424269595327, htf=-12.61700783473748
J=14.0, eb=2.143424269595327, htf=-12.614166914304441
J=14.0, eb=2.143424269595327, htf=-12.611334041788702
J=14.0, eb=2.143424269595327, htf=-12.60850917172171
J=14.0, eb=2.143424269595327, htf=-12.605692259019163
J=14.0, eb=2.143424269595327, htf=-12.602883258976671
J=14.0, eb=2.143424269595327, htf=-12.600082127265523
J=14.0, eb=2.143424269595327, htf=-12.597288819928476
J=14.0, eb=2.143424269595327, htf=-12.594503293375613
J=14.0, eb=2.143424269595327, htf=-12.591725504380275
J=14.0, eb=2.143424269595327, htf=-12.588955410075023
J=14.0, eb=2.143424269595327, htf=-12.586192967947676
J=14.0, eb=2.143424269595327, htf=-12.583438135837396
J=14.0, eb=2.143424269595327, htf=-12.58069087193082
J=14.0, eb=2.143424269595327, ht

J=14.0, eb=2.143424269595327, htf=-12.263608541948077
J=14.0, eb=2.143424269595327, htf=-12.261612523869914
J=14.0, eb=2.143424269595327, htf=-12.259620481907223
J=14.0, eb=2.143424269595327, htf=-12.257632400250388
J=14.0, eb=2.143424269595327, htf=-12.255648263183893
J=14.0, eb=2.143424269595327, htf=-12.253668055085582
J=14.0, eb=2.143424269595327, htf=-12.25169176042592
J=14.0, eb=2.143424269595327, htf=-12.249719363767268
J=14.0, eb=2.143424269595327, htf=-12.247750849763147
J=14.0, eb=2.143424269595327, htf=-12.245786203157532
J=14.0, eb=2.143424269595327, htf=-12.243825408784142
J=14.0, eb=2.143424269595327, htf=-12.24186845156573
J=14.0, eb=2.143424269595327, htf=-12.2399153165134
J=14.0, eb=2.143424269595327, htf=-12.2379659887259
J=14.0, eb=2.143424269595327, htf=-12.23602045338896
J=14.0, eb=2.143424269595327, htf=-12.234078695774599
J=14.0, eb=2.143424269595327, htf=-12.232140701240471
J=14.0, eb=2.143424269595327, htf=-12.23020645522919
J=14.0, eb=2.143424269595327, htf=-1

{'J0.0eb14.143319868324545htf-19.172368036976454': (0.0065024784141220334,
  2.8242101496744647e-06),
 'J0.0eb14.143319868324545htf-18.073755738876944': (0.019256998982398,
  8.15062454797559e-06),
 'J0.0eb14.143319868324545htf-17.56293010567955': (0.031688185690705535,
  1.307455230811268e-05),
 'J0.0eb14.143319868324545htf-17.22645785962693': (0.04380817967415326,
  1.7625896430054246e-05),
 'J0.0eb14.143319868324545htf-16.97514342191462': (0.05562852171859489,
  2.183212429694417e-05),
 'J0.0eb14.143319868324545htf-16.774472717021062': (0.0671601889147573,
  2.571848806719618e-05),
 'J0.0eb14.143319868324545htf-16.60741862292649': (0.07841362865927756,
  2.930822299100443e-05),
 'J0.0eb14.143319868324545htf-16.464317769854414': (0.0893987902240207,
  3.262272609573159e-05),
 'J0.0eb14.143319868324545htf-16.339154617469003': (0.10012515409418796,
  3.568171736771449e-05),
 'J0.0eb14.143319868324545htf-16.227928972927376': (0.11060175925704911,
  3.8503385321503465e-05),
 'J0.0eb14.14

In [ ]:
get_correlation_times(varied_prms.htf_list, varied_prms.J_list, varied_prms.eb_list, grid_prms, sim_prms.T, N=shift_compensation.N_expr, file=file)

#### Analytical calculation for time sweep

In [ ]:
for i in range(161):
    file_name = 'TimeSweep/TimeSweep_6Sites_NoPadding_NonPeriodic_SteadyState_InfiniteBath_Occupancy_2024-05-19_'+f'{i}'
    file = os.path.join(os.path.realpath('__file__'), f'../../Data/{file_name}')

    grid_prms, sim_prms, varied_prms = read_metadata(file)
    shift_compensation               = read_files_shift_compensation(file)

    get_steady_states(varied_prms.htf_list, varied_prms.J_list, varied_prms.eb_list, grid_prms, file=file)
    get_means_and_vars(varied_prms.htf_list, varied_prms.J_list, varied_prms.eb_list, grid_prms, sim_prms.T, N=shift_compensation.N_expr, limit=False, file=file)
    get_means_and_vars(varied_prms.htf_list, varied_prms.J_list, varied_prms.eb_list, grid_prms, sim_prms.T, N=shift_compensation.N_expr, limit=True, file=file)

Means could not be loaded.
Correlation times could not be loaded.
J=0.0, eb=14.143319868324545, htf=-16.0
J=0.0, eb=14.143319868324545, htf=-15.5
J=0.0, eb=14.143319868324545, htf=-15.0
J=0.0, eb=14.143319868324545, htf=-14.5
J=0.0, eb=14.143319868324545, htf=-14.0
J=0.0, eb=14.143319868324545, htf=-13.5
J=0.0, eb=14.143319868324545, htf=-13.0
J=0.0, eb=14.143319868324545, htf=-12.5
J=0.0, eb=14.143319868324545, htf=-12.0
J=3.5, eb=11.140108819624526, htf=-16.0
J=3.5, eb=11.140108819624526, htf=-15.5
J=3.5, eb=11.140108819624526, htf=-15.0
J=3.5, eb=11.140108819624526, htf=-14.5
J=3.5, eb=11.140108819624526, htf=-14.0
J=3.5, eb=11.140108819624526, htf=-13.5
J=3.5, eb=11.140108819624526, htf=-13.0
J=3.5, eb=11.140108819624526, htf=-12.5
J=3.5, eb=11.140108819624526, htf=-12.0
J=7.0, eb=8.23962329975328, htf=-16.0
J=7.0, eb=8.23962329975328, htf=-15.5
J=7.0, eb=8.23962329975328, htf=-15.0
J=7.0, eb=8.23962329975328, htf=-14.5
J=7.0, eb=8.23962329975328, htf=-14.0
J=7.0, eb=8.239623299753

J=7.0, eb=8.23962329975328, htf=-13.0
J=7.0, eb=8.23962329975328, htf=-12.5
J=7.0, eb=8.23962329975328, htf=-12.0
J=10.5, eb=5.352949615690948, htf=-16.0
J=10.5, eb=5.352949615690948, htf=-15.5
J=10.5, eb=5.352949615690948, htf=-15.0
J=10.5, eb=5.352949615690948, htf=-14.5
J=10.5, eb=5.352949615690948, htf=-14.0
J=10.5, eb=5.352949615690948, htf=-13.5
J=10.5, eb=5.352949615690948, htf=-13.0
J=10.5, eb=5.352949615690948, htf=-12.5
J=10.5, eb=5.352949615690948, htf=-12.0
J=14.0, eb=2.4541905752958177, htf=-16.0
J=14.0, eb=2.4541905752958177, htf=-15.5
J=14.0, eb=2.4541905752958177, htf=-15.0
J=14.0, eb=2.4541905752958177, htf=-14.5
J=14.0, eb=2.4541905752958177, htf=-14.0
J=14.0, eb=2.4541905752958177, htf=-13.5
J=14.0, eb=2.4541905752958177, htf=-13.0
J=14.0, eb=2.4541905752958177, htf=-12.5
J=14.0, eb=2.4541905752958177, htf=-12.0
J=0.0, eb=14.143319868324545, htf=-16.0
J=0.0, eb=14.143319868324545, htf=-15.5
J=0.0, eb=14.143319868324545, htf=-15.0
J=0.0, eb=14.143319868324545, htf=-14

J=0.0, eb=14.143319868324545, htf=-12.0
J=3.5, eb=11.140108819624526, htf=-16.0
J=3.5, eb=11.140108819624526, htf=-15.5
J=3.5, eb=11.140108819624526, htf=-15.0
J=3.5, eb=11.140108819624526, htf=-14.5
J=3.5, eb=11.140108819624526, htf=-14.0
J=3.5, eb=11.140108819624526, htf=-13.5
J=3.5, eb=11.140108819624526, htf=-13.0
J=3.5, eb=11.140108819624526, htf=-12.5
J=3.5, eb=11.140108819624526, htf=-12.0
J=7.0, eb=8.23962329975328, htf=-16.0
J=7.0, eb=8.23962329975328, htf=-15.5
J=7.0, eb=8.23962329975328, htf=-15.0
J=7.0, eb=8.23962329975328, htf=-14.5
J=7.0, eb=8.23962329975328, htf=-14.0
J=7.0, eb=8.23962329975328, htf=-13.5
J=7.0, eb=8.23962329975328, htf=-13.0
J=7.0, eb=8.23962329975328, htf=-12.5
J=7.0, eb=8.23962329975328, htf=-12.0
J=10.5, eb=5.352949615690948, htf=-16.0
J=10.5, eb=5.352949615690948, htf=-15.5
J=10.5, eb=5.352949615690948, htf=-15.0
J=10.5, eb=5.352949615690948, htf=-14.5
J=10.5, eb=5.352949615690948, htf=-14.0
J=10.5, eb=5.352949615690948, htf=-13.5
J=10.5, eb=5.35294

J=14.0, eb=2.4541905752958177, htf=-12.0
J=0.0, eb=14.143319868324545, htf=-16.0
J=0.0, eb=14.143319868324545, htf=-15.5
J=0.0, eb=14.143319868324545, htf=-15.0
J=0.0, eb=14.143319868324545, htf=-14.5
J=0.0, eb=14.143319868324545, htf=-14.0
J=0.0, eb=14.143319868324545, htf=-13.5
J=0.0, eb=14.143319868324545, htf=-13.0
J=0.0, eb=14.143319868324545, htf=-12.5
J=0.0, eb=14.143319868324545, htf=-12.0
J=3.5, eb=11.140108819624526, htf=-16.0
J=3.5, eb=11.140108819624526, htf=-15.5
J=3.5, eb=11.140108819624526, htf=-15.0
J=3.5, eb=11.140108819624526, htf=-14.5
J=3.5, eb=11.140108819624526, htf=-14.0
J=3.5, eb=11.140108819624526, htf=-13.5
J=3.5, eb=11.140108819624526, htf=-13.0
J=3.5, eb=11.140108819624526, htf=-12.5
J=3.5, eb=11.140108819624526, htf=-12.0
J=7.0, eb=8.23962329975328, htf=-16.0
J=7.0, eb=8.23962329975328, htf=-15.5
J=7.0, eb=8.23962329975328, htf=-15.0
J=7.0, eb=8.23962329975328, htf=-14.5
J=7.0, eb=8.23962329975328, htf=-14.0
J=7.0, eb=8.23962329975328, htf=-13.5
J=7.0, eb=8

C:\Users\tmija\AppData\Local\Temp\ipykernel_18232\1201438814.py:61: RuntimeWarning: invalid value encountered in sqrt
  stds[-1].append(np.sqrt(sigma_squared))


...

J=3.5, eb=11.140108819624526, htf=-12.5
J=3.5, eb=11.140108819624526, htf=-12.0
J=7.0, eb=8.23962329975328, htf=-16.0
J=7.0, eb=8.23962329975328, htf=-15.5
J=7.0, eb=8.23962329975328, htf=-15.0
J=7.0, eb=8.23962329975328, htf=-14.5
J=7.0, eb=8.23962329975328, htf=-14.0
J=7.0, eb=8.23962329975328, htf=-13.5
J=7.0, eb=8.23962329975328, htf=-13.0
J=7.0, eb=8.23962329975328, htf=-12.5
J=7.0, eb=8.23962329975328, htf=-12.0
J=10.5, eb=5.352949615690948, htf=-16.0
J=10.5, eb=5.352949615690948, htf=-15.5
J=10.5, eb=5.352949615690948, htf=-15.0
J=10.5, eb=5.352949615690948, htf=-14.5
J=10.5, eb=5.352949615690948, htf=-14.0
J=10.5, eb=5.352949615690948, htf=-13.5
J=10.5, eb=5.352949615690948, htf=-13.0
J=10.5, eb=5.352949615690948, htf=-12.5
J=10.5, eb=5.352949615690948, htf=-12.0
J=14.0, eb=2.4541905752958177, htf=-16.0
J=14.0, eb=2.4541905752958177, htf=-15.5
J=14.0, eb=2.4541905752958177, htf=-15.0
J=14.0, eb=2.4541905752958177, htf=-14.5
J=14.0, eb=2.4541905752958177, htf=-14.0
J=14.0, eb=2.

J=14.0, eb=2.4541905752958177, htf=-13.0
J=14.0, eb=2.4541905752958177, htf=-12.5
J=14.0, eb=2.4541905752958177, htf=-12.0
J=0.0, eb=14.143319868324545, htf=-16.0
J=0.0, eb=14.143319868324545, htf=-15.5
J=0.0, eb=14.143319868324545, htf=-15.0
J=0.0, eb=14.143319868324545, htf=-14.5
J=0.0, eb=14.143319868324545, htf=-14.0
J=0.0, eb=14.143319868324545, htf=-13.5
J=0.0, eb=14.143319868324545, htf=-13.0
J=0.0, eb=14.143319868324545, htf=-12.5
J=0.0, eb=14.143319868324545, htf=-12.0
J=3.5, eb=11.140108819624526, htf=-16.0
J=3.5, eb=11.140108819624526, htf=-15.5
J=3.5, eb=11.140108819624526, htf=-15.0
J=3.5, eb=11.140108819624526, htf=-14.5
J=3.5, eb=11.140108819624526, htf=-14.0
J=3.5, eb=11.140108819624526, htf=-13.5
J=3.5, eb=11.140108819624526, htf=-13.0
J=3.5, eb=11.140108819624526, htf=-12.5
J=3.5, eb=11.140108819624526, htf=-12.0
J=7.0, eb=8.23962329975328, htf=-16.0
J=7.0, eb=8.23962329975328, htf=-15.5
J=7.0, eb=8.23962329975328, htf=-15.0
J=7.0, eb=8.23962329975328, htf=-14.5
J=7.0

J=0.0, eb=14.143319868324545, htf=-14.5
J=0.0, eb=14.143319868324545, htf=-14.0
J=0.0, eb=14.143319868324545, htf=-13.5
J=0.0, eb=14.143319868324545, htf=-13.0
J=0.0, eb=14.143319868324545, htf=-12.5
J=0.0, eb=14.143319868324545, htf=-12.0
J=3.5, eb=11.140108819624526, htf=-16.0
J=3.5, eb=11.140108819624526, htf=-15.5
J=3.5, eb=11.140108819624526, htf=-15.0
J=3.5, eb=11.140108819624526, htf=-14.5
J=3.5, eb=11.140108819624526, htf=-14.0
J=3.5, eb=11.140108819624526, htf=-13.5
J=3.5, eb=11.140108819624526, htf=-13.0
J=3.5, eb=11.140108819624526, htf=-12.5
J=3.5, eb=11.140108819624526, htf=-12.0
J=7.0, eb=8.23962329975328, htf=-16.0
J=7.0, eb=8.23962329975328, htf=-15.5
J=7.0, eb=8.23962329975328, htf=-15.0
J=7.0, eb=8.23962329975328, htf=-14.5
J=7.0, eb=8.23962329975328, htf=-14.0
J=7.0, eb=8.23962329975328, htf=-13.5
J=7.0, eb=8.23962329975328, htf=-13.0
J=7.0, eb=8.23962329975328, htf=-12.5
J=7.0, eb=8.23962329975328, htf=-12.0
J=10.5, eb=5.352949615690948, htf=-16.0
J=10.5, eb=5.35294

J=10.5, eb=5.352949615690948, htf=-14.5
J=10.5, eb=5.352949615690948, htf=-14.0
J=10.5, eb=5.352949615690948, htf=-13.5
J=10.5, eb=5.352949615690948, htf=-13.0
J=10.5, eb=5.352949615690948, htf=-12.5
J=10.5, eb=5.352949615690948, htf=-12.0
J=14.0, eb=2.4541905752958177, htf=-16.0
J=14.0, eb=2.4541905752958177, htf=-15.5
J=14.0, eb=2.4541905752958177, htf=-15.0
J=14.0, eb=2.4541905752958177, htf=-14.5
J=14.0, eb=2.4541905752958177, htf=-14.0
J=14.0, eb=2.4541905752958177, htf=-13.5
J=14.0, eb=2.4541905752958177, htf=-13.0
J=14.0, eb=2.4541905752958177, htf=-12.5
J=14.0, eb=2.4541905752958177, htf=-12.0
Means could not be loaded.
Correlation times could not be loaded.
J=0.0, eb=14.143319868324545, htf=-16.0
J=0.0, eb=14.143319868324545, htf=-15.5
J=0.0, eb=14.143319868324545, htf=-15.0
J=0.0, eb=14.143319868324545, htf=-14.5
J=0.0, eb=14.143319868324545, htf=-14.0
J=0.0, eb=14.143319868324545, htf=-13.5
J=0.0, eb=14.143319868324545, htf=-13.0
J=0.0, eb=14.143319868324545, htf=-12.5
J=0.0

J=7.0, eb=8.23962329975328, htf=-15.5
J=7.0, eb=8.23962329975328, htf=-15.0
J=7.0, eb=8.23962329975328, htf=-14.5
J=7.0, eb=8.23962329975328, htf=-14.0
J=7.0, eb=8.23962329975328, htf=-13.5
J=7.0, eb=8.23962329975328, htf=-13.0
J=7.0, eb=8.23962329975328, htf=-12.5
J=7.0, eb=8.23962329975328, htf=-12.0
J=10.5, eb=5.352949615690948, htf=-16.0
J=10.5, eb=5.352949615690948, htf=-15.5
J=10.5, eb=5.352949615690948, htf=-15.0
J=10.5, eb=5.352949615690948, htf=-14.5
J=10.5, eb=5.352949615690948, htf=-14.0
J=10.5, eb=5.352949615690948, htf=-13.5
J=10.5, eb=5.352949615690948, htf=-13.0
J=10.5, eb=5.352949615690948, htf=-12.5
J=10.5, eb=5.352949615690948, htf=-12.0
J=14.0, eb=2.4541905752958177, htf=-16.0
J=14.0, eb=2.4541905752958177, htf=-15.5
J=14.0, eb=2.4541905752958177, htf=-15.0
J=14.0, eb=2.4541905752958177, htf=-14.5
J=14.0, eb=2.4541905752958177, htf=-14.0
J=14.0, eb=2.4541905752958177, htf=-13.5
J=14.0, eb=2.4541905752958177, htf=-13.0
J=14.0, eb=2.4541905752958177, htf=-12.5
J=14.0, 